In [1]:
from json import dumps, loads
import time
import threading 
import ast
import unittest
import requests
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient
G = setup()[0]


In [3]:
init_transaction = surge.init()
init_token1 = init_transaction['transaction']['outputs'][0]
init_token2 = init_transaction['transaction']['outputs'][1]
init_token3 = init_transaction['transaction']['outputs'][2]
init_token4 = init_transaction['transaction']['outputs'][3]
init_token5 = init_transaction['transaction']['outputs'][4]
init_token6 = init_transaction['transaction']['outputs'][5]
global_client.process_transaction(init_transaction)

inputs: ()
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 2}", "{\"type\": \"InitToken\", \"location\": 2}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {}}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}


<Response [200]>

In [4]:
# Create prosumer clients for locations 0 and 1
c00 = surge.SurgeClient(5000, init_token1) # location=0
c01 = surge.SurgeClient(5000, init_token2) # location=0
c10 = surge.SurgeClient(5001, init_token3) # location=1
c11 = surge.SurgeClient(5001, init_token4) # location=1
# Create sreps for locations 0 and 1 which will be clients in location 2
rep0 = surge.SREPClient(5002, init_token5) # location=2
rep1 = surge.SREPClient(5002, init_token6) # location=2

# Elect rep1 as SREP for location 0
votes = (c00.cast_srep_vote(rep0.pub), c01.cast_srep_vote(rep0.pub))
rep0.create_srep_client(5000, votes) 

# Elect rep2 as SREP for location 1
votes = (c10.cast_srep_vote(rep1.pub), c11.cast_srep_vote(rep1.pub))
rep1.create_srep_client(5001, votes) 


inputs: ('{"type": "InitToken", "location": 0}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_4b87d3a7997fc917d1ee75144f6ea37fff3814460efa85d87732a781676eda38"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd03b2ecc092ef73c5cbc9f8d423bf1d6ba6c357be98c3ec6cacc9710daf", "92c71d002bd6bf2b68ff9ce73ea9cd121403d2fefa78c5c203aca77cef8be72fc2c71d002b8cbd649a0b28445282e743bdea2385934933f0451f4100704f16a6af"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03b2ecc092ef73c5cbc9f8d423bf1d6ba6c357be98c3ec6cacc9710daf\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03b2ecc092ef73c5cbc9f8d423bf1d6ba6c357be98c3ec6cacc9710daf\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03b2ecc092ef73c5cbc9f8d423bf1d6ba6c357be98c3ec6cacc9710daf\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surg

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
inputs: ('{"type": "VoteSlipToken", "location": 0, "pub": "c7220292cd02c9bd0234dfc5a33ae3d1c7eafb13972a17f1e9c6f664d0f7c9b03a68b8c608"}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_bf16efc261e702328f1b43ad6182aa62968919e62bbeb74e72ead94f6de6ef3a"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b1fd179e097708592b898e696b8e96ddcca51604ba30a631ba534eac8c71d002be1090caa870ec50c6c1cd0b170ca2884d4502b28e66553e7470f575b"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02c5c7c0d17b8593c1d3e4a9a5a26c3d326989170830b784eecd856cd2\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd0234dfc5a33ae3d1c7eafb13972a17f1e9c6f664d0f7c9b03a68b8c608\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0234dfc5a33ae3d1c7eafb13972a17f1e9c6f664d0f7c9b03a68b8c608\"}"], "returns": [], "dependencies"

In [5]:
t00 = threading.Thread(target=c00.submit_bid, args=('EBBuy',50)) 
t01 = threading.Thread(target=c01.submit_bid, args=('EBSell',55)) 
trep0 = threading.Thread(target=rep0.accept_bids) 

t10 = threading.Thread(target=c10.submit_bid, args=('EBBuy',55)) 
t11 = threading.Thread(target=c11.submit_bid, args=('EBSell',50)) 
trep1 = threading.Thread(target=rep1.accept_bids) 


In [6]:
t00.start()
t01.start()
trep0.start()
t10.start()
t11.start()
trep1.start()

t00.join()
t01.join()
trep0.join()
t10.join()
t11.join()
trep1.join()


inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd03b2ecc092ef73c5cbc9f8d423bf1d6ba6c357be98c3ec6cacc9710daf"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 1, "pub": "c7220292cd02c9bd02346e5f6c0a5baed85fab4c12ca58167f0f4225c29f8380f548daa3a6"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd0234dfc5a33ae3d1c7eafb13972a17f1e9c6f664d0f7c9b03a68b8c608"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 1, "pub": "c7220292cd02c9bd03a379a264996aa846c707894cfebba547a058dfb1e0fc7ec8d92c6298"}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_89b7aa7a1519da8f7e22ae82afb5f22fbaf2bb5c718f4f5cc3b488d1356830bb"], "methodID": "submit_bid_proof", "parameters": ["EBBuy", "92c71d002b8eea8aaab13a12c5689d43113327b9ed532a9b956cd605b2da0c0fefc71d002b9d581c426c2fd20d7ea347e5a8577461bc485f23e9fd8865a66d66dc"], "outputs": ["{\"bid_type\": 

POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_73986a086069534421b0a8cd9d928f69ad7cb957962d085fd1e787102e06d968"], "methodID": "submit_bid", "parameters": [55, "92c71d002bc2efdc19d8b371064e8ab2bd31cbefeb777f573a0f6a6cbe4cbe0e47c71d002be1476968fc303b2c889e929d0cab1519e7ecb135b74cbeb1458c9d93"], "outputs": ["{\"quantity_sig\": \"92c71d002b0cc85c373e32b7bfe946d6723a3f9f82b1fbc9123a59195f7f7df11ac71d002b5f1fd32e6f7ab66025931d51ad05ba9625d592847d4edce8f627e97d\", \"type\": \"EBSell\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0234dfc5a33ae3d1c7eafb13972a17f1e9c6f664d0f7c9b03a68b8c608\", \"quantity\": 55}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_73986a086069534421b0a8cd9d928f69ad7cb957962d085fd1e787102e06d968": "{\"bid_type\": \"EBSell\", \"type\": \"BidProof\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0234dfc5a33ae3d1c7eafb13972a17f1e9c6f664d0f7c9b03a68b8c608\", \"quantity_sig\"

In [12]:
# '{}|{}'.format(50, loads(r1.ebtoken)['pub'])
# global_client.fix_json("{'type':'EBBuy','location':0")
global_client = ChainspaceClient(port=5001)
global_client.get_objects({'location':1})
# print buy_bids

# print loads(objs[0])['type']
# bid_proofs = rep1.client.get_objects({ 'type':'EBBuy'})
# print init_token1


http://127.0.0.1:5001/api/1.0/objects?status=1
HTTP/1.1 200 OK


['{"type": "SurgeClient", "location": 1, "pub": "c7220292cd02c9bd02346e5f6c0a5baed85fab4c12ca58167f0f4225c29f8380f548daa3a6"}',
 '{"type": "SurgeClient", "location": 1, "pub": "c7220292cd02c9bd03a379a264996aa846c707894cfebba547a058dfb1e0fc7ec8d92c6298"}',
 '{"type": "VoteSlipToken", "location": 1, "pub": "c7220292cd02c9bd02346e5f6c0a5baed85fab4c12ca58167f0f4225c29f8380f548daa3a6"}',
 '{"type": "VoteSlipToken", "location": 1, "pub": "c7220292cd02c9bd03a379a264996aa846c707894cfebba547a058dfb1e0fc7ec8d92c6298"}',
 '{"type": "SREPClient", "location": 1, "pub": "c7220292cd02c9bd03aea7ff067f5d8ad40a2242e89a340630bf38d6935c2cc78a6338f490"}',
 '{"type": "VoteSlipToken", "location": 1, "pub": "c7220292cd02c9bd03aea7ff067f5d8ad40a2242e89a340630bf38d6935c2cc78a6338f490"}',
 '{"type": "EBToken", "location": 1, "pub": "c7220292cd02c9bd02346e5f6c0a5baed85fab4c12ca58167f0f4225c29f8380f548daa3a6"}',
 '{"type": "EBToken", "location": 1, "pub": "c7220292cd02c9bd03a379a264996aa846c707894cfebba547a058dfb1